Importing the required libraries

In [130]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Retreiving the URL and creating a Beautiful soup object

In [33]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url)
soup = BeautifulSoup(req.content)

Creating the dataframe of neighborhoods and postal codes

In [102]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Ignoring cells with a borough that is Not assigned

In [104]:
df=df[df.Borough!='Not assigned']

Grouping the neighborhoods based on their postal code

In [122]:
df=df.groupby('PostalCode',axis=0).sum()

Replacing the neighborhood with Not assigned values to their borough name.

In [126]:
df['Neighborhood'].replace('Not assigned',df.Borough,inplace=True)
df.reset_index(inplace=True)

In [129]:
df.shape

(103, 3)

Getting the geographical coordinates of each postal code.

In [142]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data


In [144]:
long_lat=pd.read_csv("Geospatial_Coordinates.csv")


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [146]:
result = pd.concat([df, long_lat['Latitude'],long_lat['Longitude']], axis=1, join='inner')
result

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
